The dataset is used for this competition is synthetic, but based on a real dataset and generated using a CTGAN. The original dataset deals with predicting the category on an eCommerce product given various attributes about the listing. Although the features are anonymized, they have properties relating to real-world features.



Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Read

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
train

In [ ]:
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")
test

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv")
submission

Analyse target

In [ ]:

import matplotlib.pyplot as plt

train.groupby('target').target.count().plot.bar(ylim=0)
plt.show()


In [ ]:
target = train.groupby('target').target.count()
target

Define target

In [ ]:
target = train.target

Drop target

In [ ]:
train.drop('target', axis = 1, inplace = True)
train

Combine train and test

In [ ]:
combi = train.append(test)
combi

Drop id

In [ ]:
combi.drop('id', axis = 1, inplace = True)
combi

Check for null values

In [ ]:
combi.isnull().sum().sum()

Normalise data

In [ ]:
combi = (combi.max() - combi) / (combi.max() - combi.min())
combi

One hot encode target

In [ ]:
target = pd.get_dummies(target)
target

Define independant and dependant variables

In [ ]:
y = target
X = combi[: len(train)]
X_test = combi[len(train) :]

Split training set

In [ ]:
#split train set for testing
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=1, stratify=y)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape

Select model

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

model = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=1, max_iter=10000)).fit(X_train, y_train)
model.score(X_train, y_train)

Predict on validation set

In [ ]:
y_pred = model.predict(X_val)
model.score(X_val, y_val)

Predict on test sdet

In [ ]:
predictions = model.predict(X_test)
predictions.shape

In [ ]:
predictions[predictions < 0] = 0

Prepare submission

In [ ]:
submission.Class_1 = predictions[:, 0]
submission.Class_2 = predictions[:, 1]
submission.Class_3 = predictions[:, 2]
submission.Class_4 = predictions[:, 3]
submission.Class_5 = predictions[:, 4]
submission.Class_6 = predictions[:, 5]
submission.Class_7 = predictions[:, 6]
submission.Class_8 = predictions[:, 7]
submission.Class_9 = predictions[:, 8]

In [ ]:
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission